In [1]:
from dataclasses import dataclass, field
from vi import Agent, Config, Simulation, Vector2
import random

pygame 2.6.1 (SDL 2.28.4, Python 3.12.3)
Hello from the pygame community. https://www.pygame.org/contribute.html


In [4]:
@dataclass
class FlockingConfig(Config):
    alignment_weight: float = 1.1
    cohesion_weight: float = 0.03
    separation_weight: float = 3.1
    wind_weight: float = 0.1
    wind_direction: Vector2 = field(default_factory=lambda: Vector2(1, 0))

    def update_wind_direction(self, mutation_range = 1.0):
        dx = random.uniform(-mutation_range, mutation_range)
        dy = random.uniform(-mutation_range, mutation_range)
        self.wind_direction += Vector2(dx, dy)
        self.wind_direction = self.wind_direction.normalize()

class FlockingAgent(Agent):
    def update_wind_tolerance(self):
        move_norm = self.move.normalize() if self.move.length() > 0 else Vector2(0, 0)

        alignment = move_norm.dot(self.config.wind_direction)

        # update tolerance:
        # we increase it if moving against the wind (alignment < 0)
        # we decrease it slightly if moving with the wind (alignment > 0)
        if alignment < 0:
            self.wind_tolerance += abs(alignment) * 0.01  # learning rate when resisting wind
        else:
            self.wind_tolerance -= alignment * 0.0001       # decay rate when going with wind

        self.wind_tolerance = max(0.0, min(1.0, self.wind_tolerance))

    def on_spawn(self):
        self.wind_tolerance: float = 0.001

    def change_position(self):
        self.there_is_no_escape()
        neighbors = list(self.in_proximity_accuracy())

        if not neighbors:
            self.pos += self.move 
            return
        
        separationVelocity = Vector2() # move away from nearby agents
        alignmentVelocity = Vector2() # match the average direction of nearby agents
        cohesionVelocity = Vector2() # move toward the average position of nearby agents
        num_neighbors = len(neighbors)

        for neighbor, distance in neighbors: # loop trough each nearby agent.
            offset = self.pos - neighbor.pos # a vector pointing away from the neighbor
            if distance > 0:
                separationVelocity += offset / (distance**1.3) # push neighbors away

            alignmentVelocity += neighbor.move # sum al neighbors movement direction
            cohesionVelocity += neighbor.pos # sum all neigbors position

        # take the average of each vector to get a single direction fro each behavior
        separationVelocity /= num_neighbors
        alignmentVelocity /= num_neighbors
        cohesionVelocity = (cohesionVelocity / num_neighbors - self.pos)

        # multiply bt weights
        separationVelocity *= self.config.separation_weight
        alignmentVelocity *= self.config.alignment_weight
        cohesionVelocity *= self.config.cohesion_weight

        # single movement vector for all behaviors
        self.move = (separationVelocity + alignmentVelocity + cohesionVelocity).normalize() * self.config.movement_speed
        wind_effect = self.config.wind_direction * self.config.wind_weight * (1.0 - self.wind_tolerance)
        self.pos += self.move + wind_effect

        self.update_wind_tolerance()  # update wind tolerance based on current movement vector
        if random.random() < 0.01: # 1% chance to change wind direction and strength
            self.config.update_wind_direction()

(
    Simulation(
        FlockingConfig(image_rotation = True, movement_speed = 1.5, radius = 50, seed = 777, duration = 10000, fps_limit = 60)
    )
    .batch_spawn_agents(100, FlockingAgent, images=["images/triangle.png"])
    .run()
)